In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
sys.path.extend(['../input/efficientnet/'])
from timm import *
from timm.models import *

In [ ]:
import torch
import torch.nn as nn
import torchvision
import csv
import time
import glob
import copy
import os
import json
import cv2
import numpy as np
import pickle
import torch.optim as optim
import torch
import torch.nn.functional as F
from torch import nn as nn
from torch.optim import lr_scheduler
from timm.models import *
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils, models, datasets
from PIL import Image
from PIL import Image, ImageEnhance, ImageOps
from torchvision.models._utils import IntermediateLayerGetter
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.ops.feature_pyramid_network import (
    FeaturePyramidNetwork,
    LastLevelMaxPool,
)
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.ops.misc import FrozenBatchNorm2d
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.ops import misc as misc_nn_ops
from torch import Tensor, Size
from torch.jit.annotations import List, Optional, Tuple
from torch.nn.parameter import Parameter

    

class Linear(nn.Linear):
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        if torch.jit.is_scripting():
            bias = self.bias.to(dtype=input.dtype) if self.bias is not None else None
            return F.linear(input, self.weight.to(dtype=input.dtype), bias=bias)
        else:
            return F.linear(input, self.weight, self.bias)
    
    
    
    
class backboneNet_efficient(nn.Module):
    def __init__(self):
        super(backboneNet_efficient, self).__init__()
#         norm_layer=FrozenBatchNorm2d
        net = create_model('tf_efficientnet_b4_ns', pretrained=False)       
#         layers_to_train = ['blocks']       
#         for name, parameter in net.named_parameters():
#             if all([not name.startswith(layer) for layer in layers_to_train]):
#                 parameter.requires_grad_(False)
        self.num_features = 1792
        self.conv_stem = net.conv_stem
        self.bn1 = net.bn1
        self.act1 = net.act1       
        self.block0 = net.blocks[0]
        self.block1 = net.blocks[1]
        self.block2 = net.blocks[2]
        self.block3 = net.blocks[3]
        self.block4 = net.blocks[4]
        self.block5 = net.blocks[5]
        self.block6 = net.blocks[6]       
        self.conv_head = net.conv_head
        self.bn2 = net.bn2
        self.act2 = net.act2
        self.global_pool =  net.global_pool
        self.drop_rate = 0.4
        
        
        self.rg_cls = Linear(self.num_features, 1, bias=True)
        self.cls_cls = Linear(self.num_features, 5, bias=True)
        self.ord_cls = Linear(self.num_features, 4, bias=True)
    def forward(self, x):
        
        x1 = self.conv_stem(x)
        x2 = self.bn1(x1)
        x3 = self.act1(x2)
        x4 = self.block0(x1)
        x5 = self.block1(x4)
        x6 = self.block2(x5)
        x7 = self.block3(x6)
        x8 = self.block4(x7)
        x9 = self.block5(x8)
        x10 = self.block6(x9)       
        x11 = self.conv_head(x10)
        x12 = self.bn2(x11)
        x13 = self.act2(x12)
        x14  = self.global_pool(x13)
        if self.drop_rate > 0.:
            x14 = F.dropout(x14, p=self.drop_rate, training=self.training)
            
#         x15 = self.dense1(x14)
#         x15 = self.act2(x15)
        
#         x16 = self.dense1(x14)
#         x16 = self.act2(x16)
        
#         x17 = self.dense1(x14)
#         x17 = self.act2(x17)
            
        x15 = self.rg_cls(x14)
        x16 = self.cls_cls(x14)
        x17 = self.ord_cls(x14)

        return x15, x16, x17
    
    
    


In [ ]:

def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)


class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, flatten=False):
        super(GeM,self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps
        self.flatten = flatten
    def forward(self, x):
        x = gem(x, p=self.p, eps=self.eps)
        if self.flatten:
            x = x.flatten(1)
        return x
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'
    
    
class ThreeStage_Model(nn.Module):
    def __init__(self, backbone=None):
        super(ThreeStage_Model, self).__init__()
        
        self.backbone = tf_efficientnet_b4_ns(pretrained=False)
        self.backbone.global_pool = GeM(flatten=True)

        self.classifier = nn.Sequential(
                nn.SiLU(),
                nn.Linear(1000, 500),
                nn.SiLU(),
                nn.Linear(500, 5),
            )
        
        self.regressor = nn.Sequential(
                nn.SiLU(),
                nn.Linear(1000, 500),
                nn.SiLU(),
                nn.Linear(500, 1),
            )

        self.ordinal = nn.Sequential(
                nn.SiLU(),
                nn.Linear(1000, 500),
                nn.SiLU(),
                nn.Linear(500, 4),
            )
        
        self.final_regressor = nn.Sequential(
                nn.SiLU(),
                nn.Linear(10, 1),
            )

    def forward(self, x, final=False):
        x = self.backbone(x)
        
        c_out = self.classifier(x)
        r_out = self.regressor(x)
        o_out = self.ordinal(x)
        
        if final:
            out = torch.cat((c_out, r_out, o_out), 1)
            out = self.final_regressor(out)
            out = torch.sigmoid(out) * 4.5
            return out
        else:
            r_out = torch.sigmoid(r_out) * 4.5
            o_out = torch.sigmoid(o_out)
            return c_out, r_out, o_out
        
        

In [ ]:


from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
from PIL import Image
import csv
import cv2
from skimage import measure
import numpy as np



def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img







In [ ]:
class cropTo4_3(object):
    def __call__(self, image):
        w, h = image.size
        
        if (w / h) >= (4 / 3):
            new_h = h
            new_w = int(h * 4 / 3)
        else:
            new_h = int(w * 3 / 4)
            new_w = w
            
        left = (w - new_w)/2
        top = (h - new_h)/2
        right = left + new_w
        bottom = top + new_h

        return image.crop((left, top, right, bottom))

    
class trim(object):
    def __call__(self, image):
        bg = Image.new(image.mode, image.size, image.getpixel((0,0)))
        diff = ImageChops.difference(image, bg)
        diff = ImageChops.add(diff, diff, 2.0, -10)
        bbox = diff.getbbox()
        if bbox:
             return image.crop(bbox)



data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((384, 384)),
        transforms.RandomAffine(
            degrees=(-180, 180),
            scale=(0.8, 1.2),
            shear=(0.8, 1.2),
        ),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ColorJitter(brightness=(0.2), 
                               contrast=(0.2),
                               hue=(0.1),
                               saturation=(0.2),
                              ),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), 
    ]),
    'tta': transforms.Compose([
        transforms.Resize((420,420)),
        transforms.RandomAffine(
            degrees=(-180, 180),
        ),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.CenterCrop(384),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((420,420)),
        transforms.CenterCrop(384),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test_Cpixel': transforms.Compose([
        transforms.Resize((280,280)),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'tta_Cpixel': transforms.Compose([
        transforms.Resize((288,288)),
        transforms.RandomAffine(
            degrees=(-180, 180),
#             scale=(1.2, 1.2),
        ),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'bo_trans':transforms.Compose([
                trim(),
                cropTo4_3(),
                transforms.Resize((384 * 3 // 4, 384)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.384, 0.258, 0.174], std=[0.124, 0.089, 0.094]),
   ]),
}


def most_frequent(List):
    counter = 0
    num = List[0]

    for i in List:
        curr_frequency = List.count(i)
        if curr_frequency > counter:
            counter = curr_frequency
            num = i
            
        if curr_frequency == counter:
            num = (num + i) / 2
            
    num = round(num)
    return num

def Average(lst): 
  
    return  sum(lst) / len(lst)

In [ ]:
import os
import csv
import torch
import numpy as np

csv_name = 'submission.csv'
num_cv = 1


model_ft = backboneNet_efficient()
model_ft2 = backboneNet_efficient()
model_ft3 = backboneNet_efficient()
model_ft4 = backboneNet_efficient()


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


model_ft.load_state_dict(torch.load('../input/v22-pseudov2/rg_efficientnet_b4_v22_pseudov2_30',map_location='cuda:0'))
model_ft = model_ft.to(device)
model = model_ft

# model_ft2.load_state_dict(torch.load('../input/v18-pseudo3/rg_efficientnet_b4_v19_256_41_pseudo_v3_19',map_location='cuda:0'))
# model_ft2 = model_ft2.to(device)

# model_ft3.load_state_dict(torch.load('../input/v20pseudo/rg_efficientnet_b4_v20_256_31_pseudo_20',map_location='cuda:0'))
# model_ft3 = model_ft3.to(device)

# model_ft4.load_state_dict(torch.load('../input/v20pseudo/rg_efficientnet_b4_v20_256_31_pseudo_20',map_location='cuda:0'))
# model_ft4 = model_ft4.to(device)




thrs = [0.5, 1.5, 2.5, 3.5]

with open(csv_name, 'w', newline='') as csvFile:
    
    field = ['id_code', 'diagnosis']
    writer = csv.DictWriter(csvFile, field)
    writer.writeheader()
    
    test_path = '../input/aptos2019-blindness-detection/test_images/'
    allFileList = os.listdir(test_path)
    
    for file in allFileList:
        result = []
        for num in range(num_cv):
#             for model in [model_ft,model_ft2,model_ft3]:
                if os.path.isfile(test_path + file):
                    path = test_path + file
                    img = cv2.imread(path)
                    img = crop_image_from_gray(img)
                    img = Image.fromarray(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
                    img = data_transforms['test_Cpixel'](img)
    #                 if model != model_ft4:
    #                     img = data_transforms['test_Cpixel'](img)
    #                 elif model == model_ft4:
    #                     img = data_transforms['bo_trans'](img)
                    img = img.unsqueeze(0)
                    with torch.no_grad():
                        model.eval()
                        img = img.to(device)
                        outputs = model(img)
                        rg_outputs = outputs[0]
                        rg_outputs = torch.sigmoid(rg_outputs) * 4.5 


                        cls_outputs = outputs[1]
                        cls_softmax = torch.nn.Softmax(dim=1)
                        cls_outputs = cls_softmax(cls_outputs)
                        cls_preds = torch.max(cls_outputs, 1)
                        cls_predict = cls_preds[1].cpu().numpy()

                        ord_outputs = outputs[2]
                        ord_outputs = torch.sigmoid(ord_outputs)

                        rg_outputs = rg_outputs.unsqueeze(1)                    
                        rg_outputs[rg_outputs < thrs[0]] = 0
                        rg_outputs[(rg_outputs >= thrs[0]) & (rg_outputs < thrs[1])] = 1
                        rg_outputs[(rg_outputs >= thrs[1]) & (rg_outputs < thrs[2])] = 2
                        rg_outputs[(rg_outputs >= thrs[2]) & (rg_outputs < thrs[3])] = 3
                        rg_outputs[rg_outputs >= thrs[3]] = 4                                           
                        rg_preds = rg_outputs.tolist()
                        rg_predict = rg_preds[0][0]


                        ord_outputs[ord_outputs >= 0.5] = 1
                        ord_outputs[ord_outputs < 0.5] = 0
                        ord_predict = sum(ord_outputs[0].cpu().numpy())

                        predict = [rg_predict[0],cls_predict[0],ord_predict]
                        predict = [most_frequent(predict)]
                        
                        result.extend(predict)
#         print(result)
        final_predict = most_frequent(result)   
#         final_predict = round(Average(result))
#         print(final_predict)
#         if final_predict < thrs[0]:
#             final_predict = 0
#         elif final_predict < thrs[1]:
#             final_predict = 1
#         elif final_predict < thrs[2]:
#             final_predict = 2
#         elif final_predict < thrs[3]:
#             final_predict = 3
#         else:
#             final_predict = 4
        writer.writerow({'id_code': file.split('.png')[0],'diagnosis':int(final_predict)})